# Assessing worldwide university performance

## Introduction

Ranking world universities is an often controversial endeavour. Various countries may have different views about education and the role of university in society and about the meaning of education itself. Furthermore, world rankings often mix institutions of non-comparable size and educational scope and use arbitrary parameters in order to obtain an overall performance score. Understanding how these rankings work and how their results can be interpreted is vital for any university who wants to gain more international visibility and attract funding from public and private institutions. This is also important for perspective students, who are faced with the important choice of a college. For this reason, this report will analyze one of the most popular rankings of universities: the Times Higher Education (THE) ranking (see also <https://www.timeshighereducation.com>). It will be shown, for instance, how the weight arbitrarily attributed to certain performance parameters can significantly change the ranking in favour of under-represented countries. Finally, a machine learning algorithm will be applied in order to cluster world universities into groups with similar characteristics. This will highlight the relationship between the various performance indicators that form the overall score.

## Reference data

The database containing the THE ranking for several years is made available on Kaggle by Myles O'Neill at the address <https://www.kaggle.com/mylesoneill/world-university-rankings>. Each university is evaluated with marks ranging from 0 to 100 for the following performance indicators: teaching quality, international outlook, research (considering the sheer volume of papers produced and the reputation of said research), citations, industry income.
The overall score $S$ is the weighted average of the previous parameters according to the following weighted sum:

$S = 0.3*$teaching $+\ 0.275*$international $+\ 0.3*$research $+\ 0.1 *$citations $+\ 0.025 *$income

Additional parameters available in the database are the number of students, the student-to-staff ratio, the percentage of international students and the female-to-male ratio.